In [ ]:
import appsettings

In [ ]:
USER = appsettings.USER
ACCOUNT = appsettings.ACCOUNT
WAREHOUSE = appsettings.WAREHOUSE
DATABASE = 'SNOWFLAKE_SAMPLE_DATA'
SCHEMA = 'TPCH_SF100'
ROWS = 10000
LEVEL = 's'
TABLE = 'LINEITEM'
ASSOCIATIONS = False
OPEN_BROWSER = False

In [ ]:
import os
import snowflake.connector
import pandas as pd
import sweetviz as sv
import gc

In [ ]:
ctx = snowflake.connector.connect(
    user=USER,
    account=ACCOUNT,
    authenticator='externalbrowser'
)

In [ ]:
ctx.execute_string(f"USE WAREHOUSE {WAREHOUSE}", return_cursors=False)

In [ ]:
# SHOW TABLES COMMAND RESULT EXAMPLE (SNOWFLAKE AS 2023-05)
#
# created_on	                name        database_name	    schema_name	                kind    (more columns...)
# 2023-03-21 16:25:59.773 -0700	MY_TABLE_1	MY_DATABASE_1	    CURATED_DATA_STAGING_TEST	TABLE
# 2023-03-21 16:23:13.464 -0700	MY_TABLE_2	MY_DATABASE_2       CURATED_DATA_STAGING_TEST	TABLE
# 2023-05-04 10:43:11.121 -0700	MY_TABLE_3	MY_DATABASE_3       CURATED_DATA_STAGING_TEST	TABLE

In [ ]:
tables = []
if(LEVEL=='s'):
    schema_cur = ctx.cursor().execute(f"SHOW TABLES IN {DATABASE}.{SCHEMA}")
    for (created_on,name,database_name,schema_name,kind,comment,cluster_by,rows,bytes,owner,retention_time,automatic_clustering,change_tracking,search_optimization,search_optimization_progress,search_optimization_bytes,is_external) in schema_cur:
        tables.append((f'{database_name}.{schema_name}.{name}',comment,rows,bytes))
else:
    ctx.execute_string(f"USE DATABASE {DATABASE}", return_cursors=False)

    table_query = f"SELECT ROW_COUNT, BYTES, COMMENT FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE' AND TABLE_SCHEMA='{SCHEMA}' AND TABLE_NAME='{TABLE}'"
    table_cur = ctx.cursor().execute(table_query)
    (rows,bytes,comment) = table_cur.fetchone()
    tables.append((f'{DATABASE}.{SCHEMA}.{TABLE}',comment,rows,bytes))

In [ ]:
if not os.path.exists('obj'):
    os.mkdir('obj')

info_arr = []

for (table_name,comment,rows,bytes) in tables:
    print(f'Loading {table_name}...')

    sheet_name = table_name.split('.')[-1]

    sample_query = f'SELECT * FROM {table_name} SAMPLE BERNOULLI ({ROWS} ROWS)'
    sample_cur = ctx.cursor().execute(sample_query)
    sample_df = pd.DataFrame.from_records(
        iter(sample_cur), columns=[x[0] for x in sample_cur.description])

    is_clustering_eligible = 'YES' if bytes / 1024 / 1024 / 1024 / 1024 >= 1 else 'NO'
    info_arr.append([table_name, comment, rows, bytes, is_clustering_eligible, len(sample_df)])

    analysis = sv.analyze(sample_df, pairwise_analysis=("on" if ASSOCIATIONS else "off"))
    analysis.show_html(f'obj/{sheet_name}.html', open_browser=OPEN_BROWSER)

    del sample_df
    gc.collect()

In [ ]:
eda_info_df = pd.DataFrame(info_arr, columns = ['NAME','COMMENT','TABLE ROWS','TABLE BYTES','CLUSTERING ELIGIBLE','SAMPLE ROWS'])
excel_writer = pd.ExcelWriter(f'obj/{SCHEMA}_EDA_INFO.xlsx', engine='xlsxwriter')
eda_info_df.to_excel(excel_writer, sheet_name=SCHEMA, index=False)
worksheet = excel_writer.sheets[SCHEMA]
for idx, col in enumerate(eda_info_df):  # Loop through all columns
    series = eda_info_df[col]
    max_len = max((
        series.astype(str).str.len().max(),  # Len of largest item
        len(str(series.name))  # Len of column name/header
        )) + 9  # Adding a little extra space
    worksheet.set_column(idx, idx, max_len)  # Set column width
excel_writer.close()